# `Python to MySQL: Hiccups you will encounter`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# `Outcome:`

+ `Problems with Date-Time`
+ `Handling NaN`
+ `Schema Considerations`
+ `Dot Notation Naming`

In [232]:
import pandas as pd # dataframe stuff
# import json # 
from collections import defaultdict # dictionary where values are a list
import numpy as np # formatting data with Nan
from datetime import date
import datetime

# `Install: mysql.connector`

`conda install -c anaconda mysql-connector-python`

`pip install mysql-connector`

or `pip3 install mysql-connector`

In [233]:
# Setup MySQL for us to send data and create a new DB

import mysql.connector as mysql # establish connection to mysql 

from mysql.connector import MySQLConnection # using Class for function call

import pandas as pd  # Create Dataframe

from python_to_myql_config import read_db_config # calling config files, (you need to make this) 
# stored user credentials I have, or use the block of code below

import csv

# `Turn on mysql server for this to work: in Mac terminal`


**`mysql.server start`**

# `If you want user` <font color=red>credential</font> `setup` <font color=red>go here</font>`:`

https://github.com/MrFuguDataScience/SendDataFrom-R-Python-to-SQL-PSQL-Mongo/blob/master/Python_to_mysql_configFiles.ipynb

**IF YOU DO NOT HAVE CREDENTIALS OR IT DOESN'T MATTER TO YOU TRY THIS:**

`
cnx = mysql.connector.connect(user='bambam', password='password',
                              host='127.0.0.1',
                              database='cowboy_doodah')
cnx.close()
`

+ Remember: replace, user, password and database with your information!

In [234]:
db_config = read_db_config() # user credentials are mine, you will need to change this line

conn = MySQLConnection(**read_db_config()) # establish connection

mycursor = conn.cursor() # cursor, allows us to do queries,send/retrieve data

In [235]:
# Show What DB we are using:

# show what database we are currently using:
mycursor.execute("SELECT DATABASE()")
mycursor.fetchall()

[('project_practice',)]

In [236]:
# Turn this on, if you have issues sending data

mycursor.execute('SET GLOBAL local_infile=1')

# This checks if you have issues with exporting later due to security:
mycursor.execute('SHOW VARIABLES LIKE "secure_file_priv"')
mycursor.fetchall()

[('secure_file_priv', '')]

In [279]:
# Check Db Using:
mycursor.execute("SHOW DATABASES")
mycursor.fetchall()

[('information_schema',),
 ('JA_w_help',),
 ('mysql',),
 ('performance_schema',),
 ('project_practice',),
 ('sys',)]

In [237]:
# Check Tables:

mycursor.execute("SHOW TABLES")
mycursor.fetchall()

# mycursor.execute("DROP TABLE dot_notation_table_oidFix")
# mycursor.fetchall()

[('cpu_purchase',),
 ('dupl_transactions_copy',),
 ('Julie_Sales_dta',),
 ('Julie_Sales_dta_26',),
 ('Sales_Transactions',),
 ('user_info',),
 ('user_o',),
 ('users_inf',),
 ('users_info',)]

# `Ex.1) Evaluating Dates/Time Formatting MySQL`

+ This is an example of Timestamp data but we need to format it before using it

In [238]:
# Date-Time csv:
dates_df=pd.read_csv('date_time_ex.csv')
dates_df.head()

,createDate_date,dateScanned_date,finishedDate_date,modifyDate_date,pointsAwardedDate_date,purchaseDate_date
0,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
3,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
4,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12


In [239]:
# Creating proper format for dates
divide_by_thous_dts=dates_df.div(1000, axis=0)

# doing this across all date columns
convert_to_dts=divide_by_thous_dts.apply(pd.to_datetime, errors='coerce',utc=True,unit='s')

# If you need date with time unblock the , %H:%M:%S")) portion of code and it will work BUT, CHANGE SCHEMA 
# to DATETIME instead of DATE!

format_dt_df=convert_to_dts.apply(lambda x: x.dt.strftime("%Y-%m-%d"))
                                        
#                                        , %H:%M:%S"))
format_dt_df.head()

,createDate_date,dateScanned_date,finishedDate_date,modifyDate_date,pointsAwardedDate_date,purchaseDate_date
0,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03
1,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-02
2,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-02
3,2021-01-03,2021-01-03,NaN,2021-01-03,NaN,2021-01-03
4,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03


# `Here is a potential issue:` 
+ **`['finishedDate_date','dateScanned_date','modifyDate_date']`** you have `NaN` for dates 
and empty string is not working
+ *let's investigate all the dates first to make sure that this is not a fluke and needs further testing*


`What is the problem with NaN values for MySQL import/Insert?`


# `Python Nan is not the same as MySQL Null...`
+ You need to get creative and think about how you want to handle this!

In [240]:
format_dt_df.isnull().sum()

createDate_date              0
dateScanned_date             0
finishedDate_date         1411
modifyDate_date              0
pointsAwardedDate_date    1301
purchaseDate_date          458
dtype: int64

In [241]:
# Create Schema:

sql_01 ="""
CREATE TABLE date_table_Ex01(createDate_date DATE,
dateScanned_date DATE, finishedDate_date DATE, modifyDate_date DATE,
pointsAwardedDate_date DATE, purchaseDate_date DATE)"""

# or DATETIME depending on your needs, but change formatting above first

# `Send Data to MySQL from Python `

In [242]:
# Creating table as per requirement, sql is the schema from above we created

mycursor.execute(sql_01)

In [243]:
mycursor.executemany("""
INSERT INTO date_table_Ex01(createDate_date,
dateScanned_date, finishedDate_date, modifyDate_date,
pointsAwardedDate_date, purchaseDate_date ) VALUES(%s,%s,%s,%s,%s,%s)
""",list(format_dt_df.values))



ProgrammingError: 1054 (42S22): Unknown column 'nan' in 'field list'

# `Now What do we do?`

In [244]:
# Send Data: W/O Converting Dates Correctly (2 ways to do this)

# 1) 

"""
We have the option to grab data from one consistant column and use that as our source to work from to fill
NaN values. In this case I was considering using ['createDate_date']
"""

# deep copy: has own index and copy of data vs shallow copy which shares! If I don't do this I will overwrite
format_updated_df=format_dt_df.copy()

# Fill Nan with CreateDate_date Column for each row where corresponding Column has Nan
format_updated_df.apply(lambda x: x.fillna(format_updated_df.createDate_date,inplace=True))

# No more NaN but how do I know it did everything correct? Further investigation below
format_updated_df

,createDate_date,dateScanned_date,finishedDate_date,modifyDate_date,pointsAwardedDate_date,purchaseDate_date
0,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03
1,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-02
2,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-02
3,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03
4,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03
...,...,...,...,...,...,...
7376,2021-03-01,2021-03-01,2021-03-01,2021-03-01,2021-03-01,2021-03-01
7377,2021-03-01,2021-03-01,2021-03-01,2021-03-01,2021-03-01,2021-03-01
7378,2021-03-01,2021-03-01,2021-03-01,2021-03-01,2021-03-01,2020-08-17
7379,2021-03-01,2021-03-01,2021-03-01,2021-03-01,2021-03-01,2020-08-17


In [245]:
# Let's Check and see if its correct: USE Comparisons (Stckoverflow adapted, citation below)

ne_stacked=(format_dt_df != format_updated_df).stack()

changed = ne_stacked[ne_stacked]

changed.index.names = ['id', 'col']

changed

id    col                   
3     finishedDate_date         True
      pointsAwardedDate_date    True
14    finishedDate_date         True
      pointsAwardedDate_date    True
15    finishedDate_date         True
                                ... 
7379  finishedDate_date         True
      pointsAwardedDate_date    True
7380  finishedDate_date         True
      pointsAwardedDate_date    True
      purchaseDate_date         True
Length: 3170, dtype: bool

In [246]:
# stackoverflow adapted, citation below

import numpy as np

df1=format_dt_df
df2=format_updated_df

difference_locations = np.where(df1 != df2)

changed_from = df1.values[difference_locations]

changed_to = df2.values[difference_locations]

pd.DataFrame({'from': changed_from, 'to': changed_to}, index=changed.index)

from          to
id   col                                    
3    finishedDate_date       NaN  2021-01-03
     pointsAwardedDate_date  NaN  2021-01-03
14   finishedDate_date       NaN  2021-01-03
     pointsAwardedDate_date  NaN  2021-01-03
15   finishedDate_date       NaN  2021-01-03
...                          ...         ...
7379 finishedDate_date       NaN  2021-03-01
     pointsAwardedDate_date  NaN  2021-03-01
7380 finishedDate_date       NaN  2021-03-01
     pointsAwardedDate_date  NaN  2021-03-01
     purchaseDate_date       NaN  2021-03-01

[3170 rows x 2 columns]

In [247]:
# 2)

"""
This was a dummy date so MySQL would take import and you can query this to find your Nan values easy
if you want I would think about replacing with CreateDate for consistency. This will depend on what you are
needing as an end result or personal preference.
"""
new_date_nan_changed=format_dt_df.replace(np.nan,'1900-01-01',regex=True)
new_date_nan_changed.head()

,createDate_date,dateScanned_date,finishedDate_date,modifyDate_date,pointsAwardedDate_date,purchaseDate_date
0,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03
1,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-02
2,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-02
3,2021-01-03,2021-01-03,1900-01-01,2021-01-03,1900-01-01,2021-01-03
4,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03


In [248]:
'''
Send Dates: With Correct formatting for schema:

Creating table as per requirement, sql is the schema from above we created

mycursor.execute(sql_01) This doesn't need to be updated since format was correct except for our Nan we fixed

'''

In [249]:
# Send of batch of rows via DF to MySQL

mycursor.executemany("""
INSERT INTO date_table_Ex01(createDate_date,
dateScanned_date, finishedDate_date, modifyDate_date,
pointsAwardedDate_date, purchaseDate_date ) VALUES(%s,%s,%s,%s,%s,%s)
""",list(new_date_nan_changed.values))

In [280]:
mycursor.execute("SELECT * from date_table_Ex01")

# Get all results and store them 
query_to_df=mycursor.fetchall()

# Retain columns names, and store as DF: these data are now imported back from MySQL btw from line above
pd.DataFrame(query_to_df,columns=mycursor.column_names)

,createDate_date,dateScanned_date,finishedDate_date,modifyDate_date,pointsAwardedDate_date,purchaseDate_date
0,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03
1,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-02
2,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-02
3,2021-01-03,2021-01-03,1900-01-01,2021-01-03,1900-01-01,2021-01-03
4,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03
...,...,...,...,...,...,...
7376,2021-03-01,2021-03-01,1900-01-01,2021-03-01,1900-01-01,1900-01-01
7377,2021-03-01,2021-03-01,1900-01-01,2021-03-01,1900-01-01,1900-01-01
7378,2021-03-01,2021-03-01,1900-01-01,2021-03-01,1900-01-01,2020-08-17
7379,2021-03-01,2021-03-01,1900-01-01,2021-03-01,1900-01-01,2020-08-17


# `Ex.2) Other Errors That May Occur:`

+ `Dot Notation Naming`
+ `_id.oid`
+ `NaN in general`

In [251]:
# Data subset csv
df_parse=pd.read_csv('mysql_play_data.csv')

# Subset for ease of the example and schema later
ex_df_subset=df_parse.loc[:,['_id.oid','bonusPointsEarned','rewardsReceiptItemList_pointsEarned',
"needsFetchReview",'competitiveProduct','deleted','userFlaggedQuantity']]

ex_df_subset.head()

,_id.oid,bonusPointsEarned,rewardsReceiptItemList_pointsEarned,needsFetchReview,competitiveProduct,deleted,userFlaggedQuantity
0,5ff1e1eb0a720f0523000575,500.0,NaN,False,NaN,NaN,5.0
1,5ff1e1bb0a720f052300056b,150.0,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,150.0,NaN,True,NaN,NaN,1.0
3,5ff1e1f10a720f052300057a,5.0,NaN,False,NaN,NaN,3.0
4,5ff1e1ee0a7214ada100056f,5.0,NaN,False,NaN,NaN,4.0


In [252]:
# This is allowing me to take a note of our empty rows by column and assess how to handle later
ex_df_subset.isnull().sum()

_id.oid                                   0
bonusPointsEarned                      1401
rewardsReceiptItemList_pointsEarned    6454
needsFetchReview                       6568
competitiveProduct                     6736
deleted                                7372
userFlaggedQuantity                    7082
dtype: int64

In [253]:
# Preparing myself for the schema, thinking about our variables
ex_df_subset.dtypes

_id.oid                                 object
bonusPointsEarned                      float64
rewardsReceiptItemList_pointsEarned    float64
needsFetchReview                        object
competitiveProduct                      object
deleted                                 object
userFlaggedQuantity                    float64
dtype: object

In [254]:
# Articifically change column names so we can get the point for the example!
ex_df_subset.columns=['_id.oid', 'bonus.PointsEarned', 'rewards.ReceiptItemList_pointsEarned',
       'needs.FetchReview', 'competitive.Product', 'deleted',
       'user.FlaggedQuantity']

ex_df_subset.columns

Index(['_id.oid', 'bonus.PointsEarned', 'rewards.ReceiptItemList_pointsEarned',
       'needs.FetchReview', 'competitive.Product', 'deleted',
       'user.FlaggedQuantity'],
      dtype='object')

# `Let's see what happens if you try to send this to MySQL`

+ First create a schema and table to send these data
+ Investigate the error we get if any

In [208]:
# Create Schema and Table:

sql_02a ="""
CREATE TABLE dot_notation_table_01( _id.oid VARCHAR(255), bonus.PointsEarned FLOAT, 
rewards.ReceiptItemList_pointsEarned FLOAT, needs.FetchReview BOOL , 
competitive.Product TEXT, deleted BOOL, user.FlaggedQuantity INT )"""


In [209]:
# Send this off to MySQL

# Creating table as per requirement, sql is the schema from above we created

mycursor.execute(sql_02a)

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '.oid VARCHAR(255), bonus.PointsEarned FLOAT, 
rewards.ReceiptItemList_pointsEarn' at line 1

# `2A.) Dot Notation from JSON data, explain & fix formatting`

In [255]:
# This is MySQL problem for importing so I changed dot notation of columns
ex_df_subset.columns=ex_df_subset.columns.str.replace('.','_')

ex_df_subset

,_id_oid,bonus_PointsEarned,rewards_ReceiptItemList_pointsEarned,needs_FetchReview,competitive_Product,deleted,user_FlaggedQuantity
0,5ff1e1eb0a720f0523000575,500.0,NaN,False,NaN,NaN,5.0
1,5ff1e1bb0a720f052300056b,150.0,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,150.0,NaN,True,NaN,NaN,1.0
3,5ff1e1f10a720f052300057a,5.0,NaN,False,NaN,NaN,3.0
4,5ff1e1ee0a7214ada100056f,5.0,NaN,False,NaN,NaN,4.0
...,...,...,...,...,...,...,...
7376,603d0b710a720fde1000042a,NaN,NaN,NaN,NaN,NaN,NaN
7377,603cf5290a720fde10000413,NaN,NaN,NaN,NaN,NaN,NaN
7378,603ce7100a7217c72c000405,25.0,NaN,NaN,NaN,NaN,NaN
7379,603ce7100a7217c72c000405,25.0,NaN,NaN,NaN,NaN,NaN


In [256]:
sql_02b ="""
CREATE TABLE dot_notation_table_02( _id_oid VARCHAR(255), bonus_PointsEarned FLOAT, 
rewards_ReceiptItemList_pointsEarned FLOAT,
needs_FetchReview BOOL, competitive_Product TEXT, deleted BOOL, user_FlaggedQuantity INT )"""

In [257]:
# Creating table as per requirement, sql is the schema from above we created

mycursor.execute(sql_02b)

In [214]:
mycursor.executemany("""
INSERT INTO dot_notation_table_02(_id_oid, bonus_PointsEarned, 
rewards_ReceiptItemList_pointsEarned, needs_FetchReview, 
competitive_Product, deleted, user_FlaggedQuantity ) VALUES(%s,%s,%s,%s,%s,%s,%s)
""",list(ex_df_subset.values))

ProgrammingError: 1054 (42S22): Unknown column 'nan' in 'field list'

In [258]:
# We need to take care of NaN values but, we should do some investigation first

ex_df_subset.isnull().sum()

_id_oid                                    0
bonus_PointsEarned                      1401
rewards_ReceiptItemList_pointsEarned    6454
needs_FetchReview                       6568
competitive_Product                     6736
deleted                                 7372
user_FlaggedQuantity                    7082
dtype: int64

In [259]:
ex_df_subset['bonus_PointsEarned']=ex_df_subset['bonus_PointsEarned'].fillna(0.0)
ex_df_subset['bonus_PointsEarned']=pd.to_numeric(ex_df_subset['bonus_PointsEarned'])


ex_df_subset['rewards_ReceiptItemList_pointsEarned']=ex_df_subset['rewards_ReceiptItemList_pointsEarned'].fillna(0.0)

ex_df_subset['rewards_ReceiptItemList_pointsEarned']=pd.to_numeric(ex_df_subset['rewards_ReceiptItemList_pointsEarned'])

# 88% NaN
set(ex_df_subset['needs_FetchReview'])
ex_df_subset['needs_FetchReview'].isnull().sum()/len(ex_df_subset['needs_FetchReview'])
# Replace NaN with False
ex_df_subset["needs_FetchReview"].fillna(False,inplace=True)


# Checking to see proportion of data NaN to decide how to handle these
ex_df_subset['competitive_Product'].isnull().sum()/len(ex_df_subset['competitive_Product'])

ex_df_subset['competitive_Product']=ex_df_subset['competitive_Product'].fillna("Unknown")

set(ex_df_subset['deleted'])
#Bool values here, I replaced NaN with False as a judgement call and easier to import
ex_df_subset["deleted"].fillna(False,inplace=True)

# ex_df_subset['userFlaggedQuantity']
ex_df_subset['user_FlaggedQuantity']=ex_df_subset['user_FlaggedQuantity'].fillna(0)

ex_df_subset['user_FlaggedQuantity']=pd.to_numeric(ex_df_subset['user_FlaggedQuantity'],downcast='signed')
# ex_df_subset['userFlaggedQuantity']


In [260]:
mycursor.executemany("""
INSERT INTO dot_notation_table_02(_id_oid, bonus_PointsEarned, 
rewards_ReceiptItemList_pointsEarned, needs_FetchReview, 
competitive_Product, deleted, user_FlaggedQuantity ) VALUES(%s,%s,%s,%s,%s,%s,%s)
""",list(ex_df_subset.values))

In [261]:
mycursor.execute("SELECT * from dot_notation_table_02")

# Get all results and store them 
mm=mycursor.fetchall()

# Retain columns names, and store as DF:
pd.DataFrame(mm,columns=mycursor.column_names)

,_id_oid,bonus_PointsEarned,rewards_ReceiptItemList_pointsEarned,needs_FetchReview,competitive_Product,deleted,user_FlaggedQuantity
0,5ff1e1eb0a720f0523000575,500.0,0.0,0,Unknown,0,5
1,5ff1e1bb0a720f052300056b,150.0,0.0,0,Unknown,0,0
2,5ff1e1bb0a720f052300056b,150.0,0.0,1,Unknown,0,1
3,5ff1e1f10a720f052300057a,5.0,0.0,0,Unknown,0,3
4,5ff1e1ee0a7214ada100056f,5.0,0.0,0,Unknown,0,4
...,...,...,...,...,...,...,...
7376,603d0b710a720fde1000042a,0.0,0.0,0,Unknown,0,0
7377,603cf5290a720fde10000413,0.0,0.0,0,Unknown,0,0
7378,603ce7100a7217c72c000405,25.0,0.0,0,Unknown,0,0
7379,603ce7100a7217c72c000405,25.0,0.0,0,Unknown,0,0


# `2B.) _id.OID issues explained`

+ Why is this an issue anyway?

In [262]:
# I artifically reset this so we can play and see what happens!
ex_df_subset=ex_df_subset.rename(columns={'_id_oid':'_id.oid'})
ex_df_subset.head()

,_id.oid,bonus_PointsEarned,rewards_ReceiptItemList_pointsEarned,needs_FetchReview,competitive_Product,deleted,user_FlaggedQuantity
0,5ff1e1eb0a720f0523000575,500.0,0.0,False,Unknown,False,5
1,5ff1e1bb0a720f052300056b,150.0,0.0,False,Unknown,False,0
2,5ff1e1bb0a720f052300056b,150.0,0.0,True,Unknown,False,1
3,5ff1e1f10a720f052300057a,5.0,0.0,False,Unknown,False,3
4,5ff1e1ee0a7214ada100056f,5.0,0.0,False,Unknown,False,4


In [263]:
sql_02b_01 ="""
CREATE TABLE dot_notation_table_oidFix( _id.oid VARCHAR(255), bonus_PointsEarned FLOAT, 
rewards_ReceiptItemList_pointsEarned FLOAT,
needs_FetchReview BOOL, competitive_Product TEXT, deleted BOOL, user_FlaggedQuantity INT )"""

In [264]:
mycursor.execute(sql_02b_01)

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '.oid VARCHAR(255), bonus_PointsEarned FLOAT, 
rewards_ReceiptItemList_pointsEarn' at line 1

In [268]:
# OID issue and convert:

# Changed this name _id.oid because of convention naming systems for DB and I don't want confusion/errors
ex_df_subset=ex_df_subset.rename(columns={ex_df_subset.columns[0]:"OID"})

In [269]:
sql_02b ="""
CREATE TABLE dot_notation_table_oidFix( OID VARCHAR(255), bonus_PointsEarned FLOAT, 
rewards_ReceiptItemList_pointsEarned FLOAT,
needs_FetchReview BOOL, competitive_Product TEXT, deleted BOOL, user_FlaggedQuantity INT )"""

In [270]:
# Creating table as per requirement, sql is the schema from above we created

mycursor.execute(sql_02b)

In [273]:
mycursor.executemany("""
INSERT INTO dot_notation_table_oidFix( OID, bonus_PointsEarned, 
rewards_ReceiptItemList_pointsEarned, needs_FetchReview, 
competitive_Product, deleted, user_FlaggedQuantity ) VALUES(%s,%s,%s,%s,%s,%s,%s)
""",list(ex_df_subset.values))

In [275]:
mycursor.execute("SELECT * from dot_notation_table_oidFix")

# Get all results and store them 
mm=mycursor.fetchall()

# Retain columns names, and store as DF:
pd.DataFrame(mm,columns=mycursor.column_names)

,OID,bonus_PointsEarned,rewards_ReceiptItemList_pointsEarned,needs_FetchReview,competitive_Product,deleted,user_FlaggedQuantity
0,5ff1e1eb0a720f0523000575,500.0,0.0,0,Unknown,0,5
1,5ff1e1bb0a720f052300056b,150.0,0.0,0,Unknown,0,0
2,5ff1e1bb0a720f052300056b,150.0,0.0,1,Unknown,0,1
3,5ff1e1f10a720f052300057a,5.0,0.0,0,Unknown,0,3
4,5ff1e1ee0a7214ada100056f,5.0,0.0,0,Unknown,0,4
...,...,...,...,...,...,...,...
7376,603d0b710a720fde1000042a,0.0,0.0,0,Unknown,0,0
7377,603cf5290a720fde10000413,0.0,0.0,0,Unknown,0,0
7378,603ce7100a7217c72c000405,25.0,0.0,0,Unknown,0,0
7379,603ce7100a7217c72c000405,25.0,0.0,0,Unknown,0,0


# Like, Share & <font color=red>SUB</font>scribe

# `Citations & Help:`

# ◔̯◔

https://stackoverflow.com/questions/24942538/sql-wont-insert-null-values-with-bulk-insert

https://stackoverflow.com/questions/45756557/inserting-nan-value-into-mysql-database

https://www.tutorialspoint.com/can-mysql-automatically-convert-empty-strings-to-null

https://stackoverflow.com/questions/41027882/convert-nan-values-to-null-in-mysql-columns-with-python-code

https://dev.mysql.com/doc/refman/8.0/en/problems-with-null.html

https://www.tutorialspoint.com/can-mysql-automatically-convert-empty-strings-to-null

https://www.programiz.com/python-programming/datetime/strftime

https://stackoverflow.com/questions/66477410/how-can-i-insert-id-oid-5ec796a7868d607e00f4b18d-and-time-15901

https://stackoverflow.com/questions/59817317/how-to-convert-multiple-columns-datatype-format-to-another-datatype-formats-at-a

https://stackoverflow.com/questions/29177498/python-pandas-replace-nan-in-one-column-with-value-from-corresponding-row-of-sec

https://stackoverflow.com/questions/28901683/pandas-get-rows-which-are-not-in-other-dataframe

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html

https://stackoverflow.com/questions/17095101/compare-two-dataframes-and-output-their-differences-side-by-side

https://www.pythonfixing.com/2022/04/fixed-how-to-properly-insert-pandas-nat.html